In [1]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.8/719.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 41.4 MB/s eta 0:00:00
Mounted at /content/gdrive


In [4]:
from fastbook import *

In [5]:
np.random.seed(5555)

In [7]:
pwd

'/content'

In [9]:
# path =  ("C:\\Users\\Oranuch\\Documents\\Python project\\shoe")
path = ("/content/gdrive/MyDrive/Shoeprint")
path

'/content/gdrive/MyDrive/Shoeprint'

In [10]:
fnames = get_image_files(path)
fnames

(#39) [Path('/content/gdrive/MyDrive/Shoeprint/Levis/Le_1.jpg'),Path('/content/gdrive/MyDrive/Shoeprint/Levis/Le_2.jpg'),Path('/content/gdrive/MyDrive/Shoeprint/Nike/N_1.jpg'),Path('/content/gdrive/MyDrive/Shoeprint/Nike/N_2.jpg'),Path('/content/gdrive/MyDrive/Shoeprint/Nike/N_3.jpg'),Path('/content/gdrive/MyDrive/Shoeprint/Nike/N_4.JPG'),Path('/content/gdrive/MyDrive/Shoeprint/Nike/N_5.JPG'),Path('/content/gdrive/MyDrive/Shoeprint/Nike/N_7.JPG'),Path('/content/gdrive/MyDrive/Shoeprint/Nike/N_8.JPG'),Path('/content/gdrive/MyDrive/Shoeprint/Nike/N_9.JPG')...]

In [ ]:
pat = r"([^/]+)_\d+.*$"
re.findall(pat, str(fnames[0]))

['C:\\Users\\Oranuch\\Documents\\Python project\\shoe\\A']

In [ ]:
item_tfms = RandomResizedCrop(460, min_scale=0.8, ratio=(1., 1.))

In [ ]:
batch_tfms = [*aug_transforms(size=224, max_warp=0.15), Normalize.from_stats(*imagenet_stats)]

In [ ]:
aug_transforms??
RandomResizedCrop??

Init signature:
RandomResizedCrop(
    self,
    size: 'int | tuple',
    min_scale: 'float' = 0.08,
    ratio=(0.75, 1.3333333333333333),
    resamples=(2, 0),
    val_xtra: 'float' = 0.14,
    max_scale: 'float' = 1.0,
    **kwargs,
)
Source:        
@delegates()
class RandomResizedCrop(RandTransform):
    "Picks a random scaled crop of an image and resize it to `size`"
    split_idx,order = None,1
    def __init__(self, 
         size:int|tuple, # Final size, duplicated if one value is specified,, 
         min_scale:float=0.08, # Minimum scale of the crop, in relation to image area
         ratio=(3/4, 4/3), # Range of width over height of the output
         resamples=(BILINEAR, NEAREST), # Pillow `Image` resample mode, resamples[1] for mask
         val_xtra:float=0.14, # The ratio of size at the edge cropped out in the validation set
         max_scale:float=1., # Maximum scale of the crop, in relation to image area
         **kwargs
    ):
        size = _process_sz(size)
     

In [ ]:
bs = 64

In [ ]:
dls = ImageDataLoaders.from_name_re(path, fnames, pat, bs=bs,
                                     batch_tfms=batch_tfms, item_tfms=item_tfms)

In [ ]:
ImageDataLoaders.from_name_re??

Signature:
ImageDataLoaders.from_name_re(
    path,
    fnames,
    pat,
    *,
    bs: 'int' = 64,
    val_bs: 'int' = None,
    shuffle: 'bool' = True,
    device=None,
)
Source:   
    @classmethod
    @delegates(DataLoaders.from_dblock)
    def from_name_re(cls, path, fnames, pat, **kwargs):
        "Create from the name attrs of `fnames` in `path`s with re expression `pat`"
        return cls.from_name_func(path, fnames, RegexLabeller(pat), **kwargs)
File:      c:\users\oranuch\anaconda3\lib\site-packages\fastai\vision\data.py
Type:      method


In [ ]:
dls.show_batch(max_n=9, figsize=(8, 9))

ValueError: This DataLoader does not contain any batches

In [ ]:
pets = DataBlock(blocks=(ImageBlock, CategoryBlock),
                 get_items=get_image_files,
                 splitter=RandomSplitter(),
                 get_y=RegexLabeller(pat=r"/([^/]+)_\d+.*"),
                 item_tfms=item_tfms,
                 batch_tfms=batch_tfms)

In [ ]:
splitter = RandomSplitter(valid_pct=0.2, seed=5555)
splitter(fnames)

((#32) [34,13,30,28,5,35,17,33,2,4...], (#7) [18,1,31,29,22,19,9])

In [ ]:
path_im = "C:\\Users\\Oranuch\\Documents\\Python project\\shoe"

In [ ]:
dls = pets.dataloaders(path_im, bs=bs)

In [ ]:
dls.show_batch(max_n=9, figsize=(8, 9))

ValueError: This DataLoader does not contain any batches

In [ ]:
dls.vocab[2]

'Le'

In [ ]:
dls.vocab

['A', 'L', 'Le', 'N']

In [ ]:
dls.vocab.o2i.get("Le")

2

In [ ]:
dls.vocab.o2i

{'A': 0, 'L': 1, 'Le': 2, 'N': 3}

In [ ]:
learn = cnn_learner(dls, resnet34, pretrained=True, metrics=error_rate).to_fp16()

C:\Users\Oranuch\anaconda3\lib\site-packages\fastai\vision\learner.py:288: UserWarning: `cnn_learner` has been renamed to `vision_learner` -- please update your code
  warn("`cnn_learner` has been renamed to `vision_learner` -- please update your code")
C:\Users\Oranuch\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Oranuch\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
learn.fit_one_cycle(20)

C:\Users\Oranuch\anaconda3\lib\site-packages\torch\amp\autocast_mode.py:202: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
C:\Users\Oranuch\anaconda3\lib\site-packages\torch\cuda\amp\grad_scaler.py:118: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


epoch,train_loss,valid_loss,error_rate,time
0,nan,5.223201,0.714286,00:11
1,nan,5.223201,0.714286,00:10
2,nan,5.223201,0.714286,00:10
3,nan,5.223201,0.714286,00:10
4,nan,5.223201,0.714286,00:10
5,nan,5.223201,0.714286,00:09
6,nan,5.223201,0.714286,00:10
7,nan,5.223201,0.714286,00:09
8,nan,5.223201,0.714286,00:09
9,nan,5.223201,0.714286,00:10


C:\Users\Oranuch\anaconda3\lib\site-packages\fastprogress\fastprogress.py:73: UserWarning: Your generator is empty.
  warn("Your generator is empty.")


In [ ]:
learn.save("01i-stage1")

In [ ]:
learn.recorder.plot_loss()

In [ ]:
learn.recorder.plot_sched()

In [ ]:
learn.show_results()

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
losses, idxs = interp.top_losses()

len(dls.valid_ds) == len(losses) == len(idxs)

In [ ]:
interp.plot_top_losses(4, figsize=(14, 10))

In [ ]:
interp.plot_confusion_matrix(figsize=(12, 12), dpi=72)

In [ ]:
interp.most_confused(min_val=3)

In [ ]:
learn.load("01i-stage1")

In [ ]:
learn.unfreeze()

In [ ]:
learn.fit_one_cycle(2)

In [ ]:
learn.load("01i-stage1")

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()

In [ ]:
learn.fit_one_cycle(3, lr_max=slice(3e-7, 3e-5))

In [ ]:
learn.save('01i-stage2')

In [ ]:
learn = cnn_learner(dls, resnet50, pretrained=True, metrics=error_rate).to_fp16()

In [ ]:
learn.fine_tune(6)